# **Weather Classification Using Transfer Learning**

### Extracting The Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive'

'Colab Notebooks'	     'Play Books Notes'
 CollegeMagazine_2023.docx   'Rajesh Covishield-II certificate.pdf'
 Education		     'Tentative Course List (Jan - April 2023).pdf'
 multiclass-weather-dataset  'Torrents '
 NM3.ipynb


### Unzipping the Dataset

In [ ]:
#!unzip '/content/multiclass-weather-dataset.zip'

### Importing the Libraries

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

### Configure Image Generator Class

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = [.99, 1.01], brightness_range = [0.8, 1.2], data_format = "channels_last", fill_mode = "constant", horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

### Apply ImageDataGenerator functionality to Train set and Test set

In [ ]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split

In [ ]:
!mkdir /content/drive/MyDrive/multiclass-weather-dataset/train
!mkdir /content/drive/MyDrive/multiclass-weather-dataset/test

# define paths to source and destination directories
source_dir = "/content/drive/MyDrive/multiclass-weather-dataset/dataset"
train_dir = "/content/drive/MyDrive/multiclass-weather-dataset/train"
test_dir = "/content/drive/MyDrive/multiclass-weather-dataset/test"

# create train and test directories for each class
class_names = os.listdir(source_dir)
for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

# split data into training and testing sets for each class
for class_name in class_names:
    class_dir = os.path.join(source_dir, class_name)
    image_files = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    train_files, test_files = train_test_split(image_files, test_size=0.2)

    # copy training images for the current class
    for filename in train_files:
        destination_path = os.path.join(train_dir, class_name, os.path.basename(filename))
        shutil.copy(filename, destination_path)

    # copy testing images for the current class
    for filename in test_files:
        destination_path = os.path.join(test_dir, class_name, os.path.basename(filename))
        shutil.copy(filename, destination_path)

mkdir: cannot create directory ‘/content/drive/MyDrive/multiclass-weather-dataset/train’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/multiclass-weather-dataset/test’: File exists


In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/multiclass-weather-dataset/train', target_size = (180, 180), batch_size = 64, class_mode = 'categorical')

Found 1446 images belonging to 5 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/multiclass-weather-dataset/test', target_size = (180, 180), batch_size = 64, class_mode = 'categorical')

Found 546 images belonging to 5 classes.


In [ ]:
IMAGE_SIZE = [224, 224]

### Pre-Trained CNN Model as a Feature Extractor

In [ ]:
VGG19 = VGG19(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
for layer in VGG19.layers:
  layer.trainable=False

In [ ]:
from zmq import XPUB
x = Flatten()(VGG19.output)

### Adding Dense Layer

In [ ]:
prediction = Dense(5, activation='softmax')(x)

In [ ]:
model = Model(inputs=VGG19.input, outputs=prediction)

### Configure the Learning process

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Train the Model

In [ ]:
r = model.fit(training_set, validation_data=test_set, epochs=50, steps_per_epoch=len(training_set), validation_steps=len(test_set))

Epoch 1/50
46/46 [==============================] - 54s 887ms/step - loss: 1.0444 - accuracy: 0.6300 - val_loss: 0.5154 - val_accuracy: 0.7875
Epoch 2/50
46/46 [==============================] - 40s 866ms/step - loss: 0.4289 - accuracy: 0.8513 - val_loss: 0.3149 - val_accuracy: 0.9011
Epoch 3/50
46/46 [==============================] - 39s 851ms/step - loss: 0.2952 - accuracy: 0.9025 - val_loss: 0.3403 - val_accuracy: 0.8846
Epoch 4/50
46/46 [==============================] - 39s 856ms/step - loss: 0.2560 - accuracy: 0.9142 - val_loss: 0.2123 - val_accuracy: 0.9304
Epoch 5/50
46/46 [==============================] - 39s 850ms/step - loss: 0.1859 - accuracy: 0.9474 - val_loss: 0.2370 - val_accuracy: 0.9304
Epoch 6/50
46/46 [==============================] - 39s 851ms/step - loss: 0.1700 - accuracy: 0.9495 - val_loss: 0.1699 - val_accuracy: 0.9505
Epoch 7/50
46/46 [==============================] - 43s 934ms/step - loss: 0.1793 - accuracy: 0.9481 - val_loss: 0.1760 - val_accuracy: 0.9524

### Checking Model Accuracy

In [ ]:
loss, accuracy = model.evaluate(test_set, steps=11, verbose=2, use_multiprocessing=True, workers=2)
print(f'Model Performance on Test Images: \nAccuracy = {accuracy} \nLoss = {loss}')

11/11 - 4s - loss: 0.0665 - accuracy: 0.9886 - 4s/epoch - 358ms/step
Model Performance on Test Images: 
Accuracy = 0.9886363744735718 
Loss = 0.06654641032218933


### Save the Model

In [ ]:
model.save('wcv.h5')

### Test the Model

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model("/content/wcv.h5")

Taking an image as input and checking the results

In [ ]:
img = image.load_img(r"/content/drive/MyDrive/multiclass-weather-dataset/test/rainy/rain116.jpg", target_size=(180,180))
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)
preds = model.predict(x)
pred = np.argmax(preds, axis=1)
index = ['cloudy','foggy','rainy','shine','sunrise']
result = str(index[pred[0]])
result

ValueError: ignored